# Execute batch groundness evaluation flow using Promptflow Python SDK 
프롬프트 플로우는 아이디어 발상, 프로토타이핑, 테스트, 평가부터 프로덕션 배포 및 모니터링에 이르기까지 LLM 기반 AI 애플리케이션의 엔드투엔드 개발 주기를 간소화하도록 설계된 개발 도구 모음입니다. 이를 통해 신속한 엔지니어링이 훨씬 쉬워지고 프로덕션 수준의 품질로 LLM 앱을 빌드할 수 있습니다. 

> ✨ ***Note*** <br>
> Promptflow는 오픈소스 프로젝트이며, Azure ML, Azure AI Studio, Local VS Code환경을 모두 지원합니다.  

## Prerequisites
3.10 또는 이후 버전의 Python 가상 환경을 구성합니다 : 
 1. 명령 Palette를 엽니다 (Ctrl+Shift+P).
 1. Python을 검색합니다: Create Environment를 선택합니다.
 1. Venv / Conda를 선택하고 새 환경을 만들 위치를 선택합니다.
 1. Python interpreter 버전을 선택합니다. 3.10 이상의 버전으로 생성합니다.

디펜던시 설치를 위해 아래 코드를 실행하여 실행에 필요한 package들을 설치하세요. 

```bash
pip install -r requirements.txt
```

## Set up your environment
현재의 repository를 Clone 하세요. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

.env-sample 파일을 기반으로 .env 파일을 만듭니다. 새 .env 파일을 노트북이 들어 있는 폴더에 복사하고 변수를 업데이트합니다.

## Steps
#### 1. Create Promptflow client with Credential and configuration
#### 2. AI Studio batch run to get the base run data 
#### 3. Run Groundedness Evaluation of the Promptflow 


In [ ]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError

load_dotenv()

with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name


In [ ]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        while(pbar.n < 3):
            pbar.update(1)
        print("Promptflow Running Completed")

## 1. Create Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


## 2. AI Studio batch run to get the base run data 

## Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
> ✨ ***important*** <br>
> Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.

In [ ]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)
file.close()
column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

base_run = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, base_run)

In [ ]:
detail = pf_azure_client.get_details(base_run)

detail

## 3. Run Groundedness Evaluation of the Promptflow 
The eval-groundness flow is illustrating measures how grounded the model's predicted answers are against the context. Even if LLM’s responses are true, if not verifiable against context, then such responses are considered ungrounded.

> 🧪 +For Your Information<br>
> **Groundedness** is a measure of how well the model's responses are grounded in the context. A grounded response is one that is directly supported by the context. For example, if the context is about a dog, a grounded response would be "Dogs are mammals." An ungrounded response would be "Dogs can fly."

In [ ]:
import datetime

eval_groundedness_flow_path = "./evaluation/"
data_path = "./data/qna_outdoor.jsonl"

with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping={
        "question": "${data.question}",
        "context": context.get("context")    ,
        "answer": "${run.outputs.gpt4o_answer}",
    }
eval_name = "eval_groundedness"
now = datetime.datetime.now()
timestamp = now.strftime("%m_%d_%H%M")
eval_name = str(eval_name + "_" + timestamp)

eval_groundedness_result = pf_azure_client.run(
    flow=eval_groundedness_flow_path,
    data=data_path,
    run=base_run,  # use run as the variant
    column_mapping=column_mapping,
    display_name=eval_name,
    name=eval_name,
)



# pf_azure_client.stream(eval_groundedness_result)

In [ ]:
monitor_status(pf_azure_client, eval_groundedness_result)

In [ ]:
detail = pf_azure_client.get_details(eval_groundedness_result)

detail